In [3]:
from hossam import load_data
from matplotlib import pyplot as plt
from matplotlib import font_manager as fm
import seaborn as sb
import numpy as np
import pandas as pd
from pandas import DataFrame
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_reset
from scipy.stats import zscore, pearsonr, spearmanr, probplot, shapiro, kstest, anderson

from statsmodels.formula.api import ols
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [4]:
origin=load_data('diamonds')
print('\n===== 데이터 크기 확인 =====')
print(f'데이터셋 크기 : {origin.shape}')
print(f'행 개수 : {origin.shape[0]}개')
print(f'열 개수 : {origin.shape[1]}개')
print('\n===== 타입 확인 =====')
print(origin.info())
origin.head()

[data] https://data.hossam.kr/data/kaggle/diamonds.xlsx
[desc] 다이아몬드를 커팅, 색상, 투명도, 가격 및 기타 속성 조사한 데이터 셋 (출처: https://www.kaggle.com/datasets/shivam2503/diamonds)

field    description
-------  -------------------------------------------------------------------
price    다이아몬드 가격 (USD, $326 ~ $18,823)
carat    중량 (0.2~5.01)
cut      컷 품질 (Fair, Good, Very Good, Premium, Ideal)
color    색상 등급 - J (worst) to D (best)
clarity  투명도 등급 (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
x        길이 mm (0~10.74)
y        너비 mm (0--58.9)
z        두께 mm (0--31.8)
depth    비율 정보 = z / mean(x, y) = 2 * z / (x + y) (43--79)
table    다이아몬드의 가장 넓은 지점에 비해 상단(테이블)의 너비 (43--95)


===== 데이터 크기 확인 =====
데이터셋 크기 : (53940, 10)
행 개수 : 53940개
열 개수 : 10개

===== 타입 확인 =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   price    53940 non-null  int64  
 1   carat    

,price,carat,cut,color,clarity,x,y,z,depth,table
0,326,0.23,Ideal,E,SI2,3.95,3.98,2.43,61.5,55.0
1,326,0.21,Premium,E,SI1,3.89,3.84,2.31,59.8,61.0
2,327,0.23,Good,E,VS1,4.05,4.07,2.31,56.9,65.0
3,334,0.29,Premium,I,VS2,4.20,4.23,2.63,62.4,58.0
4,335,0.31,Good,J,SI2,4.34,4.35,2.75,63.3,58.0


In [5]:
df=origin.drop(index=origin[(origin['x']==0) | (origin['y']==0) | (origin['z']==0)].index)

In [6]:
q1, q2, q3=df['price'].quantile([0.25, 0.5, 0.75])
print(q1)
print(q2)
print(q3)

949.0
2401.0
5323.25


In [7]:
df_q1=df[df['price']<949]
df_q2=df[(df['price']>=949) & (df['price']<2401)]
df_q3=df[(df['price']>=2401) & (df['price']<5323.25)]
df_q4=df[df['price']>=5323.25]
display(df_q1)
display(df_q2)
display(df_q3)
display(df_q4)

,price,carat,cut,color,clarity,x,y,z,depth,table
0,326,0.23,Ideal,E,SI2,3.95,3.98,2.43,61.5,55.0
1,326,0.21,Premium,E,SI1,3.89,3.84,2.31,59.8,61.0
2,327,0.23,Good,E,VS1,4.05,4.07,2.31,56.9,65.0
3,334,0.29,Premium,I,VS2,4.20,4.23,2.63,62.4,58.0
4,335,0.31,Good,J,SI2,4.34,4.35,2.75,63.3,58.0
...,...,...,...,...,...,...,...,...,...,...
53635,552,0.30,Ideal,I,VVS1,4.30,4.33,2.68,62.1,56.0
53636,552,0.30,Ideal,D,SI1,4.28,4.32,2.65,61.6,57.0
53637,552,0.30,Very Good,D,SI1,4.26,4.28,2.67,62.5,57.0
53638,552,0.30,Ideal,D,SI1,4.29,4.33,2.63,61.0,57.0


,price,carat,cut,color,clarity,x,y,z,depth,table
36651,949,0.35,Premium,J,VS1,5.67,5.64,3.53,62.4,58.0
36652,949,0.35,Premium,J,VS1,5.67,5.64,3.53,62.4,58.0
36653,949,0.40,Premium,E,VS2,4.79,4.68,2.90,61.2,60.0
36654,949,0.30,Very Good,E,IF,4.29,4.32,2.62,60.9,59.0
36655,949,0.41,Good,D,VVS2,4.71,4.73,2.99,63.3,59.0
...,...,...,...,...,...,...,...,...,...,...
51704,2400,0.70,Very Good,F,VS2,5.72,5.78,3.52,61.2,56.0
51705,2400,0.70,Very Good,F,VS2,5.79,5.85,3.43,58.9,57.0
51706,2400,0.58,Very Good,D,VVS2,5.28,5.35,3.35,63.0,56.0
51707,2400,0.71,Ideal,G,SI1,5.77,5.79,3.54,61.2,58.0


,price,carat,cut,color,clarity,x,y,z,depth,table
90,2757,0.70,Ideal,E,SI1,5.70,5.72,3.57,62.5,57.0
91,2757,0.86,Fair,E,SI2,6.45,6.33,3.52,55.1,69.0
92,2757,0.70,Ideal,G,VS2,5.70,5.67,3.50,61.6,56.0
93,2759,0.71,Very Good,E,VS2,5.68,5.73,3.56,62.4,57.0
94,2759,0.78,Very Good,G,SI2,5.81,5.85,3.72,63.8,56.0
...,...,...,...,...,...,...,...,...,...,...
53935,2757,0.72,Ideal,D,SI1,5.75,5.76,3.50,60.8,57.0
53936,2757,0.72,Good,D,SI1,5.69,5.75,3.61,63.1,55.0
53937,2757,0.70,Very Good,D,SI1,5.66,5.68,3.56,62.8,60.0
53938,2757,0.86,Premium,H,SI2,6.15,6.12,3.74,61.0,58.0


,price,carat,cut,color,clarity,x,y,z,depth,table
12760,5324,1.21,Premium,H,SI2,6.75,6.78,4.24,62.7,59.0
12761,5324,1.21,Very Good,H,SI2,6.70,6.75,4.21,62.6,61.0
12762,5324,1.21,Very Good,I,SI1,6.76,6.92,4.25,62.1,56.0
12763,5324,1.21,Ideal,J,VS1,6.81,6.86,4.26,62.3,55.0
12764,5324,0.90,Very Good,F,VS1,6.18,6.23,3.80,61.2,56.0
...,...,...,...,...,...,...,...,...,...,...
27745,18803,2.00,Very Good,H,SI1,7.95,8.00,5.01,62.8,57.0
27746,18804,2.07,Ideal,G,SI2,8.20,8.13,5.11,62.5,55.0
27747,18806,1.51,Ideal,G,IF,7.37,7.41,4.56,61.7,55.0
27748,18818,2.00,Very Good,G,SI1,7.90,7.97,5.04,63.5,56.0


In [8]:
def ols_report(tbl, data):
    xnames=[n for n in fit.model.exog_names if n!='const']
    indi_mydata1=data.filter(xnames)
    variables=[]
    for i, v in enumerate(tbl.tables[1].data):
        name=v[0].strip()
        if name not in xnames:
            continue
        j=list(indi_mydata1.columns).index(name)
        vif=variance_inflation_factor(indi_mydata1,j)

        p=float(v[4].strip())
        stars=lambda p: (
            '***' if p<0.001 else
            '**' if p<0.01 else
            '*' if p<0.05 else
            ''
        )

        variables.append({
            '종속변수':yname,
            '독립변수':name,
            'B':v[1].strip(),
            '표준오차':v[2].strip(),
            'Beta':float(fit.params[name])*(data[name].std(ddof=1)/data[yname].std(ddof=1)),
            't':'%s%s'%(v[3].strip(), stars(p)),
            'p-value':p,
            '공차':1/vif,
            'vif':vif
        })

    rdf=DataFrame(variables)

    if rdf.empty:
        return None, "데이터 없음", "모델 유의하지 않음", []

    result_dict={}
    for i in [0,2]:
        for item in tbl.tables[i].data:
            n=len(item)
            for j in range(0, n, 2):
                key=item[j].strip()[:-1]
                value=item[j+1].strip()
                if not key or not value:
                    continue
                result_dict[key]=value
    
    result_report=f'R({result_dict['R-squared']}), R²({result_dict['Adj. R-squared']}), F({result_dict['F-statistic']}), 유의확률({result_dict['Prob (F-statistic)']}), Durbin-Watson({result_dict['Durbin-Watson']})'

    tpl='%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 %s(F(%s,%s)=%s, p %s 0.05).'
    model_report=tpl%(
        rdf['종속변수'][0],
        ','.join(list(rdf['독립변수'])),
        '유의하다' if float(result_dict['Prob (F-statistic)'])<=0.05 else '유의하지 않다',
        result_dict['Df Model'],
        result_dict['Df Residuals'],
        result_dict['F-statistic'],
        '<=' if float(result_dict['Prob (F-statistic)'])<=0.05 else '>'
    )

    variable_reports=[]
    s='%s의 회귀계수는 %s(p %s 0.05)로, %s에 대하여 %s 예측변인인 것으로 나타났다.'

    for i in rdf.index:
        row=rdf.iloc[i]
        variable_reports.append(s%(row['독립변수'],
                                   row['B'],
                                   '<=' if float(row['p-value'])<0.05 else '>',
                                   row['종속변수'],
                                   '유의미한' if float(row['p-value'])<0.05 else '유의하지 않은'))
        
    return rdf, result_report, model_report, variable_reports

In [9]:
print('='*50)
print('🎓 1사분위수 선형회귀 분석')
print('='*50)
mydata=df_q1
yname='price'
# mydata1=mydata.copy()
mydata1=mydata.drop(['cut','color','clarity'], axis=1)
x=mydata1.drop(yname, axis=1)
y=mydata1[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
rdf, r, m, v=ols_report(tbl1, mydata)
display(rdf)

print('='*50)
print('🎓 2사분위수 선형회귀 분석')
print('='*50)
mydata=df_q2
yname='price'
# mydata1=mydata.copy()
mydata1=mydata.drop(['cut','color','clarity'], axis=1)
x=mydata1.drop(yname, axis=1)
y=mydata1[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
rdf, r, m, v=ols_report(tbl1, mydata)
display(rdf)

print('='*50)
print('🎓 3사분위수 선형회귀 분석')
print('='*50)
mydata=df_q3
yname='price'
# mydata1=mydata.copy()
mydata1=mydata.drop(['cut','color','clarity'], axis=1)
x=mydata1.drop(yname, axis=1)
y=mydata1[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
rdf, r, m, v=ols_report(tbl1, mydata)
display(rdf)

print('='*50)
print('🎓 4사분위수 선형회귀 분석')
print('='*50)
mydata=df_q4
yname='price'
# mydata1=mydata.copy()
mydata1=mydata.drop(['cut','color','clarity'], axis=1)
x=mydata1.drop(yname, axis=1)
y=mydata1[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
rdf, r, m, v=ols_report(tbl1, mydata)
display(rdf)

🎓 1사분위수 선형회귀 분석


,종속변수,독립변수,B,표준오차,Beta,t,p-value,공차,vif
0,price,carat,419.7738,177.105,0.139582,2.370*,0.018,0.000534,1873.702360
1,price,x,713.5430,36.114,1.051061,19.758***,0.000,0.000079,12640.615534
2,price,y,-1273.8182,54.567,-1.874816,-23.344***,0.000,0.000072,13936.756067
3,price,z,1275.0954,97.241,1.195742,13.113***,0.000,0.000037,27040.258422
4,price,depth,-65.2940,4.755,-0.503879,-13.731***,0.000,0.000099,10081.579574
5,price,table,-8.0314,0.648,-0.112400,-12.394***,0.000,0.001114,897.738758


🎓 2사분위수 선형회귀 분석


,종속변수,독립변수,B,표준오차,Beta,t,p-value,공차,vif
0,price,carat,856.8251,133.345,0.241312,6.426***,0.000,0.004727,211.538520
1,price,x,557.1449,42.197,0.500514,13.203***,0.000,0.000424,2358.026767
2,price,y,44.2309,10.248,0.045884,4.316***,0.000,0.002047,488.610886
3,price,z,20.8776,9.676,0.016750,2.158*,0.031,0.006016,166.233063
4,price,depth,-7.8341,2.537,-0.025236,-3.088**,0.002,0.001485,673.278975
5,price,table,-15.2183,1.180,-0.077467,-12.899***,0.000,0.001719,581.829038


🎓 3사분위수 선형회귀 분석


,종속변수,독립변수,B,표준오차,Beta,t,p-value,공차,vif
0,price,carat,-160.6697,239.141,-0.032719,-0.672,0.502,0.000619,1614.452838
1,price,x,-870.9142,116.386,-0.408014,-7.483***,0.000,0.000079,12678.621337
2,price,y,1969.1407,104.377,0.896717,18.866***,0.000,0.000087,11492.687542
3,price,z,1057.0357,235.140,0.312408,4.495***,0.000,0.000046,21921.670757
4,price,depth,-12.4006,13.511,-0.024002,-0.918,0.359,0.000128,7838.938418
5,price,table,-7.3413,2.329,-0.019987,-3.151**,0.002,0.001365,732.676747


🎓 4사분위수 선형회귀 분석


,종속변수,독립변수,B,표준오차,Beta,t,p-value,공차,vif
0,price,carat,3341.9065,405.945,0.356573,8.232***,0.000,0.002155,464.102073
1,price,x,996.7433,295.198,0.171117,3.377**,0.001,0.000429,2332.842271
2,price,y,-71.5508,54.487,-0.014932,-1.313,0.189,0.003048,328.033917
3,price,z,2312.2421,400.346,0.248299,5.776***,0.000,0.000169,5914.108277
4,price,depth,-278.0076,33.040,-0.110303,-8.414***,0.000,0.000419,2388.685723
5,price,table,-127.0285,10.525,-0.074278,-12.069***,0.000,0.001397,715.883816
